<a href="https://colab.research.google.com/github/daisuke1227/Test/blob/main/CoverGen_NO_UI_v2_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credits to ardha27, creator of the base notebook
https://github.com/ardha27


Fixed and improved by:
[Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851.),
[Raid](http://discord.com/users/975389927355318323.), [Hina](http://discord.com/users/444684887363026974.) & [GDR](http://discord.com/users/310840858767523846.)

Members of the Staff of [AI HUB](https://discord.gg/aihub) on Discord

### Guide to using this notebook: https://docs.google.com/document/d/e/2PACX-1vThk7Qo7yCWNVbxOmahl2R8_Jgi6TFuMBUIi-PWre_HIN0lFTq-dr37Rh5iJlGgYb_vFapXMHt2W8Kp/pub

In [ ]:
#@title # **Clone and Install**
#@markdown The installation takes about 6 minutes, if it takes much longer ping me at [AI HUB](https://discord.gg/aihub)

#Fixing CUDA bug
#!sudo apt update
#!yes | sudo DEBIAN_FRONTEND=noninteractive apt-get -yq install cuda-11-8
#!find / -name '*libcublasLt.so*'

#AICoverGen starting code
!git clone https://github.com/Eddycrack864/AICoverGen-NO-UI-en.git
%cd /content/AICoverGen-NO-UI-en
!pip install -q -r requirements.txt
!sudo apt update
!sudo apt install sox
#New code to fix the CUDA by Hina and GDR bug
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/

#Download MDXNet Vocal Separation and Hubert Base Models
!python src/download_models.py

In [ ]:
#@markdown #**Model Download Function**
#@markdown Paste your Drive or Hugging Face link
url = "https://huggingface.co/vexxed/VexxedRVC/resolve/main/Sonic-SA2-Ov2Super-40k-300e.zip?download=true"  # @param {type:"string"}
#@markdown Give a name to the model
dir_name = "sonic"  # @param {type:"string"}

import os
import zipfile
import shutil
import urllib.request
import gdown

BASE_DIR = os.getcwd()
rvc_models_dir = os.path.join(BASE_DIR, 'rvc_models')

def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder, exist_ok=True)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, dirs, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)

            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file was found in the extracted zip. Please check {extraction_folder}.')

    # move model and index file to extraction folder
    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    # remove any unnecessary nested folders
    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

def download_model(url, dir_name):
    try:
        extraction_folder = os.path.join(rvc_models_dir, dir_name)

        if not os.path.exists(extraction_folder):
            print(f'[~] Downloading voice model with name {dir_name}...')

            zip_name = url.split('/')[-1]

            urllib.request.urlretrieve(url, zip_name)

            print('[~] Extracting zip...')
            extract_zip(extraction_folder, zip_name)
            print(f'[+] {dir_name} Model successfully downloaded!')
        else:
            print(f'[~] Voice model directory {dir_name} already exists! Skipping download...')

    except Exception as e:
        raise Exception(str(e))

def download_online_model_drive(file_id, dir_name):
    try:
        extraction_folder = os.path.join(rvc_models_dir, dir_name)

        if not os.path.exists(extraction_folder):
            print(f'[~] Downloading voice model with name {dir_name}...')

            zip_name = f'{dir_name}.zip'
            gdown.download(f'https://drive.google.com/uc?id={file_id}', zip_name, quiet=False)

            print('[~] Extracting zip...')
            extract_zip(extraction_folder, zip_name)
            print(f'[+] {dir_name} Model successfully downloaded!')
        else:
            print(f'[~] Voice model directory {dir_name} already exists! Skipping download...')

    except Exception as e:
        raise Exception(str(e))

# Detecta si la URL es de Google Drive o Hugging Face
if 'drive.google.com' in url:
    file_id_drive = url.split('/')[-2]
    download_online_model_drive(file_id_drive, dir_name)
elif 'huggingface.co' in url:
    download_model(url, dir_name)
else:
    print('Unrecognized URL. Make sure you provide a Google Drive or Hugging Face link.')

In [ ]:
#@title # **Mount Drive (Optional)**
#@markdown ## This is only used if the audio for the cover is uploaded to Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title # **Generate Cover**
# @markdown ## Output generated inside "FIX\song_output\random_number"
# @markdown YT link or audio path if in Drive
SONG_INPUT = "https://www.youtube.com/watch?v=rNQy3BxEcII&pp=ygU3amFuZSByZW1vdmVyL2RsdHprIC0gd29vZHNpZGUgZ2FyZGVucyAxNiBkZWNlbWJlciAyMDEyIA%3D%3D" # @param {type:"string"}
RVC_DIRNAME = "sonic" # @param {type:"string"}
PITCH_CHANGE = 0 # @param {type:"integer"}
PITCH_CHANGE_ALL = 0 # @param {type:"integer"}
# @markdown Voice Conversion Options
INDEX_RATE = 0.75 # @param {type:"number"}
FILTER_RADIUS = 3 # @param {type:"integer"}
PITCH_DETECTION_ALGO = "rmvpe" # @param ["rmvpe", "mangio-crepe"]
CREPE_HOP_LENGTH = 64 # @param {type:"integer"}
PROTECT = 0.33 # @param {type:"number"}
REMIX_MIX_RATE = 0.25  # @param {type:"number"}
# @markdown Audio Mixing Options
MAIN_VOL = 0 # @param {type:"integer"}
BACKUP_VOL = 0 # @param {type:"integer"}
INST_VOL = 0 # @param {type:"integer"}
# @markdown Reverb Control
REVERB_SIZE = 0.15 # @param {type:"number"}
REVERB_WETNESS = 0.2 # @param {type:"number"}
REVERB_DRYNESS = 0.8 # @param {type:"number"}
REVERB_DAMPING = 0.7 # @param {type:"number"}
# @markdown Output Format
OUTPUT_FORMAT = "wav" # @param ["mp3", "wav"]

import subprocess

command = [
    "python",
    "src/main.py",
    "-i", SONG_INPUT,
    "-dir", RVC_DIRNAME,
    "-p", str(PITCH_CHANGE),
    "-k",
    "-ir", str(INDEX_RATE),
    "-fr", str(FILTER_RADIUS),
    "-rms", str(REMIX_MIX_RATE),
    "-palgo", PITCH_DETECTION_ALGO,
    "-hop", str(CREPE_HOP_LENGTH),
    "-pro", str(PROTECT),
    "-mv", str(MAIN_VOL),
    "-bv", str(BACKUP_VOL),
    "-iv", str(INST_VOL),
    "-pall", str(PITCH_CHANGE_ALL),
    "-rsize", str(REVERB_SIZE),
    "-rwet", str(REVERB_WETNESS),
    "-rdry", str(REVERB_DRYNESS),
    "-rdamp", str(REVERB_DAMPING),
    "-oformat", OUTPUT_FORMAT
]

# Open a subprocess and capture its output
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Print the output in real-time
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()

# @markdown Example of generated audio path

# @markdown ![route.png](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOoAAACuCAIAAAC3PwZjAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAArUSURBVHhe7d1NSBxZAsDxttqP+EEucxgYQkA2LHvJwSALOzGwIy4LMhOYgJBI5pDIBoPIbGKGEMIGkQTJ0jQhKsIiHgJ97BwG9LBoLg2Zg2AOggEDBryI5NADOfTBg/u+qutVdbXdxrb1rf8fDbG7qqs/6t+vXpdCEgAAAAAAAAAAAAAAAAAAAAAAAAAAoLI75t+j8O0L8wMAnGzf3lncK+fDi2/1Oi8+FG/RQ2dwy96HxZLR9M5isL7eAnAkrBBLmTKtdfxW7btFGrXTpl4cqf3yLR19i/mG72gPwNZoXjouAzUVm2ZEmXWsUIvDbDWbA2rlEPnat+uArRuYN6AODpNvOODFO8wbUGeHyzc0gwgQL+rDHj+j/Ar3Tbx0A8wbUC+HzrdkC9SLutkn3+pOJ8TMHggYdbL/yKrts4598sz6HJTdFFBLh8rXWiBH3LgTwcAROky+Qa6mVmtFBmDUwZfna421wa32TJiAcdS+NF/rttBEwbqdGQSO2hfmGzv0anbAZTcJ1ELw977+3/KWCoI061iFxo2wwd/7MgADAAAAAADANXPv9/bez5krgFvIFw4jXziMfOEw8oXDyBcOI184jHzhMPKFw8gXDiNfOIx84TDyhcPIFw4jXziMfOEw8oXDyBcOI184jHzhMPKFw8gXAAAAAAAAAACcEPzaAg4jXziMfOEw8oXDyBcOI9+aGlvO7+3ll8fM1VqbX9/b21ufN9dAvrVFvvVFvjVFvvVV73y7RzJr+cKu/H+693YL+fcLqRudZpHSNZpZ/1RQi0UHm8vpQWvxnNh7+Tep1JtNs4VP65m79t37xaK82Xh+/fXjjHh1B4lpv0e/J8sMpSNjyi/fkz+OvRELY62bN1ff/ZV8+XpBIfzk5Y4IPVX1STC7Rr7wWPk3R/VJcUN98x1b/mTe98Du9oK/Fzt/kbssrLA+36+Xxu/Fwmrqgl7amXrnl2erOt8Kj16LfAuFyDPcXrilF5PvF6lrvmoX5n+bHhvo0jd0jWbX17Ij3fra0MK22COFzddjvbLIrqGXb2UUu6vTJlC9F/0VukeyH+X11bRa+JO5d/aXXvlp6O5/urgtY6k230qPvm++vvKTB3V3seztyyH54ruHpn9TN7yb1p/dffP1MXmIkO+abVcfeuPlfzc/xInsyDgXUqv26FPIb65Ykwfdn787taFFeZsJVOdrr5BeFTeY3al27eYr+96DcotV5lvx0WuRb3iwDK1Mvg7ovJFaWNvOhw6hfgExOziyw2S+oRXspA6Zb8VHr0W+kfLU2GFmF+Trlq7+n0aeLm6Kkk001Y2+ZfM95OShutF3+9dBdUVSS2s6+u6uptTP0oXpVXEgJN+TQ36vf7/82Mx0xR7qHXut8v3viLpe1dy3bL6H/epW6dF13/lVNdsRS5fNhyOU74j6brq9oD9Cic7efwyZr37qqYr1y819U+/ktc3XI2Jp53dj5gRFJN//yHwL7+aG9HvY3T90o1f9hKPXP78e09fuZuYHs0I1Zx7K5ysc6sRZpUfXfQfUaYRIvonHuchL3PfMg/3a06HvBXLd0tH3b5lNs9QID+c4Yl2jc8vvrfO+H5cPft63TL635la386sv7a09ffv5APkK+z56IvFDavmjWZpfy4y8kk8nkm/iwuB0btt8hESDn1en9e36qarzvvrlR877ioDFBs0dP28u/zsl7xDJVzzDf2WDZ7hb2PxVH7jgulvm4K/nvp3fFY/OwXxSzx1t9Ru9ogcKIKTzaU71aiusz/lHZ4l8cYL1pxbXzVm5uJnJcSJfAAAAAAAAAAAAAIiSf6Nd8kdVgBvIFw4jXziMfOEw8oXDyBcOI184jHzhMPKFw8gXDiNfOIx84TDyhcPIFw4jXziMfOEw8oXDyBcOI184jHzhMPKFw8gXAAAAAAAAAACcEPzaAg4jXziMfOEw8oXD6pfvn5q2Zjpyl/UVbyndsTHQoH5uyDzu2HmQ7Ek0ZCc6diYa76pbtb7LzVtTHVujyWteInG5eWemw7q076Racle9PrNuFaJbkBf/aUjnv0kuPWrdmVKL0pGNq6c3mjTXStwdOGNvNriU3kW+FWey5801LXJ3/40K6bvYuDLZJleYat0YTt5uMbefXvXL96vkmmjlR9WK+tnfr97C846dIZFnNN/znY0boqTxxrtioRAXn7hsDXnhEsq6/WNMYUG+59XDhZduDXs9ZnFt8u0752Xut+4cPF/5SbZWkJfiO3Nq1XHyIEdclWkicalJvvu6VC+5MtOx9r1oKJzv2WQu1bGTak6f1ddNvvZ+7ftDckWsM9WcNjdUIBN53vTIXIsaHxIDW+vK370+Nar1fO0tjLfvzLRkvlKLq8s3dtRU1N2D+KL5BsSnqHQ7nrckX2nr0l8aLopVWhvm5WcgdOg4jeqYb3GSkLj2fcvOxJmNGZWd3FttS5fkCkG+npcV6aRaMp3ynkZJvkJ6VKRQbb7p4fbI5MQmNxVZKh+x2Nlx5tvTK1/72lUrVs9bmBQznKZxc/1UqudXN1nPM9mHGOc2BhpzehfqRL4Ry4N806NiAnoma7crlBt9HyavyWvJ3EzHyvXGtRft4r5L3Q2Tt9TheKJp0h+/ZaD3G2eHz6jZbdvWk6bZc2aRMCg+VKWjb6pp3BygTb63rzRtiIcQW55syYqR0Fcp34Ba82D5yrdupmW+1VzVzHYi79KpUs981dSzedZLzD8Qu0fWIHaSvNEc/XW+TZkhcVhsW/mruo9N5Ru+tG89bBw3dcp8g0VTqjB1KX5HzI5bK+hL+ENiii9entvDv3p61mblZaol87VZrPONXuJG64Pnq565+uSHxB2OTpd65usfi8V3NbnzxFgovsnJGac5ZKs+ijtefC9RdwrE5Cv7W+rWi1W+Pyf7RIXDYhbbtnIlkWjxcmLCbRoSs5c2sf7CJTl9TLQ0TA60yC9Dj/XgLWbbXvahnFDaG89dafAz00+vfeOmNyiGZ69hXLWuZu3S0eYrHrp02kO+Ml/b7q75IU7+d/NDnPzyPbPNsvSO6RPf1eRwK/ficFKMxMW8TB/XvUfXxQ5u3xgIn1Io2Vs9ncncMzFVaJ6VR1WZr16q+rBG9PIT1sk7InR5QCjOtrOXGnrUbOHiWW92WNTc6j+i2tSTxkF9TdDnUvwvTzrfamL6knwZfU8AWdjG/eYtPZCId3+8eeW5dXAXfeidpGOKTH9j95Y6nSwH2or5tiSunWuInCQOSlInQ9R2bF72WXF4Lv0kqJdTj3yZ+54IqoZ0m4lA7Cc1lfSjUX34h0hz0ldcLZ7aLJ+vOp1cId/0qCygOFXVgtFXbTw+X3tuc0z5cubhRJj9Wc4I/V0ud781foTyFdSpgI6tm/4UoiRfM3kw590q5Hu+W51sftY825mQGxRz30G5/Z378lyejiZu8iDvrp7AceYrYuW87/F7dFO+6f6+UUEEx8RovsL8A1Fnq6rT5BtzMV/yKuQryPNxkftavxbR/UUvwQqHzFe/2NKL6Tj+0cXFf0P4rdvx6+kTA14w8MgTscGvwWLyTZxNrqT9hkrzfdG6dit52+zCyvkK41ebN8TsRdx3qnVjtHHc/EbNuP3nxtwTP3G18X/6cR97voL9Nw9b9q/TAbfoP2bamQj95gVwxsU/JnMTrWsDnvW7PwAAAAAAAAAAAAAAAAAAAAAAAAAA/h8kEv8Dt+TF9GPllNwAAAAASUVORK5CYII=)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')